In [5]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

from nltk.tokenize import sent_tokenize
import torch 

import torch 
import rouge
import os
from sklearn.feature_extraction.text import TfidfVectorizer


In [6]:
class tfidfSummarizer():
    def __init__(self, tfidf, tfidf_matrix, kmeans=True):
        self.tfidf_matrix = tfidf_matrix
        self.tfidf = tfidf
        self.kmeans = kmeans

    def summarize(self, text):
        sentence_score_list, sentence_list = score_article(text, self.tfidf, self.tfidf_matrix, 0)
        if self.kmeans:
            summary = generate_summary_kmeans(sentence_score_list, sentence_list)
        else:
            summary = generate_summary_max(sentence_score_list, sentence_list, alpha=1.15)
        return summary


def score_sentence(sentence, tfidf, tfidf_matrix, a_index_):
  # Calculate the sum tfidf score of words as sentence score
  # Take in a sentence (string), and the article's index in the corpus (for word lookup)

  sentence_score = 0
  sentence = sentence.replace(' ,', '').strip()

  # For each word in a sentence, look up its score from tfidf matrix, return 0 if lookup fails
  for i_word in sentence.split(' '):
    #print(i_word)
    sentence_score += tfidf_matrix[a_index_, tfidf.vocabulary_.get(i_word.lower(), 10)]
  
  return sentence_score


def score_article(article, tfidf, tfidf_matrix, a_index_):
  # Generate a list of scores of each sentence of an article
  # Take in an article (list of sentences), and the article's index in the corpus (for word lookup)

  sentence_score_list = []
  sentence_list = article.split('.')

  # For each sentence in an article, call score_sentence and append output to sentence_score_list
  for i_sentence in sentence_list:
    sentence_score = score_sentence(i_sentence, tfidf, tfidf_matrix, a_index_)
    sentence_score_list.append(sentence_score)

  return sentence_score_list, sentence_list


def generate_summary_max(sentence_score_list, sentence_list, alpha=1.15):
  # Generate a summary from sentences that have score alpha times larger than the article' sentence mean score
  # Take in a list of sentence scores, list of sentences, thresholding multiplier alpha

  sentence_score_array = np.asarray(sentence_score_list)
  mean_score = np.mean(sentence_score_array)
  sentence_list_arr = np.asarray(sentence_list, dtype=object)
  summary = sentence_list_arr[sentence_score_array > alpha*mean_score]
  summary = ' '.join(summary)

  return summary

def generate_summary_kmeans(sentence_score_list, sentence_list, k=4):
    sentence_score_list = np.expand_dims(np.array(sentence_score_list), axis=1)

    if len(sentence_list) < k:
        k = len(sentence_list)

    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(np.array(sentence_score_list))
    label_max_tfidf = kmeans.labels_[np.argmax(sentence_score_list)]
    idxes = np.where(kmeans.labels_ == label_max_tfidf)[0]
    
    summary = ''
    for idx in idxes:
        summary += sentence_list[idx]
        
    return summary

In [7]:
def compute_tfidf(data_path):
    article_list = []
    for file in os.listdir(data_path):
        #print(file)
        with open(f'{data_path}/{file}', 'r') as f:
            article = f.read().splitlines()
            article_list.append(article[4:-1])   # leave out titles, abstracts
            f.close()

    for i in range(len(article_list)):
        article_list[i] = list(filter(None, article_list[i]))   # Remove blank spaces
        article_list[i] = ' '.join(article_list[i])

    # Fit Tfidf
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(article_list)
    return tfidf, tfidf_matrix

In [15]:
viet_in = '''Khởi_tố kẻ_trộm hơn 1 tạ thóc và hơn 8 triệu đồng của chú ruột để lấy tiền mua ma_tuý

Ngày 27/3 , Cơ_quan Cảnh_sát điều_tra Công_an TP. Hưng_Yên , tỉnh Hưng_Yên cho biết , đơn_vị vừa ra quyết_định khởi_tố vụ án , khởi_tố bị_can đối_với đối_tượng Mai_Văn_Thương ( SN 1989 , trú tại đội 11 , thôn An_Chiểu 1 , xã Liên_Phương , TP. Hưng_Yên ) để điều_tra về hành_vi trộm_cắp tài_sản .
Theo tài_liệu điều_tra của cơ_quan công_an , vào_khoảng 7h30 ngày 13/3 , lợi_dụng gia_đình ông Mai_Văn_Thịnh ( chú ruột đối_tượng Thương ) ở cạnh nhà đi vắng , đối_tượng này đã đạp gãy chấn_song cửa_sổ , đột_nhập vào nhà ông Thịnh trộm_cắp 121kg thóc mang bán cho người cùng thôn lấy 700.000 đ .
Không dừng lại , sau đó đối_tượng tiếp_tục quay lại lục_soát tủ nhà ông Thịnh trộm_cắp 8.500.000 đ tiền_mặt ( ông Thịnh để dưới đáy tủ ) , rồi dùng số tiền trên để đi mua ma_tuý về sử_dụng và tiêu_xài hết 6.080.000 đ .
Đến ngày 15/3 , đối_tượng Thương đã đến Cơ_quan điều_tra Công_an TP. Hưng_Yên tự_thú và khai nhận toàn_bộ hành_vi phạm_tội của mình , đồng_thời giao_nộp cho cơ_quan công_an 3.120.000 đ .
Hiện Công_an TP. Hưng_Yên đã thu_giữ toàn_bộ 121kg thóc đối_tượng đã trộm_cắp để trao_trả cho gia_đình ông Thịnh .
Được biết Thương là đối_tượng nghiện ma_tuý từ nhiều năm nay , đã có 1 tiền_án về tội Tàng_trữ trái_phép chất ma_tuý bị TAND tỉnh Hưng_Yên xử_phạt 2 năm 3 tháng tù_giam .
Ra tù năm 2016 , đối_tượng này tiếp_tục có hành_vi cố_ý gây thương_tích , bị Công_an TP. Hưng_Yên ra quyết_định xử_phạt 2,5 triệu đồng .
Vụ án đang được Công_an TP. Hưng_Yên hoàn_thiện hồ_sơ để xử_lý Mai_Văn_Thương theo quy_định của pháp_luật .

Đối_tượng Mai_Văn_Thương tại cơ_quan công_an .'''


viet_in_2 = '''Hồi còn nhỏ, ngây ngô nghĩ rằng, một mối quan hệ, nếu bị khoảng cách chia xa, thể nào cũng sẽ tàn phai. Thế nhưng nếu không xa nhau, thì tình cảm cũng sẽ còn mãi. Thế là hồi đó, chia tay bạn bè cấp 2, tôi buồn hết 2 năm cấp 3. Cảm giác của một đứa không có gì bám víu giữa thành phố xa lạ nhưng lại không chịu chấp nhận một ai xung quanh. Riết rồi khó chia sẻ đến tận bây giờ. Vốn tôi nghĩ, thật tình những vấn đề của bản thân, chẳng ai hiểu nổi. Một đứa trẻ 15 tuổi, bảo là vấn đề to lớn như phá sản, tang tóc,... thật không phải. Nhưng vấn đề có nhỏ không? Đáp án là không! Thay đổi môi trường sống thật ra chỉ là giọt nước tràn li cho rất nhiều những vấn đề khác nhau, khó có lối ra, chỉ toàn nút thắt. Có chăng chỉ là giống sợi chỉ đỏ, khâu mà có nút chỗ nào, thì đành cắt đi khâu lại ở nút đó. Cứ thế bản thân vừa muốn kết nối, nhưng cũng tạo cho mình một rào cản khó có thể xóa bỏ. Nói là chia sẻ, nhưng chỉ dừng lại ở mặt chữ, là chia sẻ câu chuyện, trên nền chất liệu phong cách ngôn ngữ sinh hoạt, chứ lời nói chẳng có tí cảm xúc nào. Riết thành quen, bây giờ lời nói buông ra, cũng chẳng còn biết làm thế nào cho đằm thắm thứ cảm xúc đang bồn chồn muốn bộc phát trong tim. 
Lớn lên một chút, thôi thì môi trương thay đổi nhiều, chí ít đã quen những biến thiên hàng ngày của cuộc sống. Sống mà không có sự thay đổi, thì còn gọi gì là cuộc sống. Mãi bình an vui vẻ, chỉ có trên thiên đàng, mãi đau khổ sợ hãi, chỉ ở dưới địa ngục. Ba tầng trên thế gian, chắc nhân thế là hỗn loạn nhất! Bao nhiêu hỉ nộ ái ố đều phải kinh qua thì mới là người. Thế nhưng có lẽ cũng vì thế mà tôi vỡ òa, thật ra con người ta ở cạnh nhau mà lòng vẫn xa nhau được đấy thôi! Cái gọi là "cô đơn trong chính mối quan hệ của mình" thì giữa người nào cũng có. Người ta kết nối về mặt địa lí thì dễ dàng, chứ kết nối về mặt tâm hồn, e khó! Những con người của hôm nay, chúng tôi đối mặt nhau, ngồi cạnh nhau mà ngỡ là "một vòng trái đất". Người kia nghĩ gì, tôi không biết, cũng dần cảm thấy không còn muốn biết. Thành ra hai bên tự xây tường thành bao bọc cảm xúc. Bây giờ mình ôm nhau một cái được không? Để chất dẫn truyền thần kinh sinh ra làm ta hưng phấn? Tôi lại một lần nữa e khó! 
Bất cứ mối quan hệ nào, tôi cho rằng đều giống như 2 người đang đi trên dây. Khi người kia muốn tiến đến thì người còn lại phải đứng im và ngược lại, thành ra cứ mãi chệch nhịp. Còn nếu cả 2 cùng di chuyển được, thì phải có một cái gọi là "ĐỒNG TÂM". Suy nghĩ và cảm xúc cùng đồng điệu, chứ không cả 2 ngã nhào ngay! 
Nói chung suy đi xét lại, bí quyết để duy trì mối quan hệ tốt là gì? Không có quy luật nào cả. Làm sao mà để tất cả các phân số trên đời có chung mẫu số được? Vô lí. Rồi một người tiến một người lùi, cũng không xong, nếu muốn gần nhau hơn, chỉ có là cùng nhau hiểu về đối phương. Cái hiểu này không chỉ hiểu đối phương ở hiện tại, mà còn hiểu ở quá khứ. Từ đó mà chia sẻ và thấu hiểu. '''

In [13]:
# RUN SCRIPT 
path = '../../../vietnews/data/test_tokenized'
tfidf, tfidf_matrix = compute_tfidf(path)

In [17]:
summarizer = tfidfSummarizer(tfidf, tfidf_matrix, kmeans=True)
summary = summarizer.summarize(viet_in_2)
summary

' Thế nhưng có lẽ cũng vì thế mà tôi vỡ òa, thật ra con người ta ở cạnh nhau mà lòng vẫn xa nhau được đấy thôi! Cái gọi là "cô đơn trong chính mối quan hệ của mình" thì giữa người nào cũng có'